## Goal of this notebook
Previously, we have built models on the training data, and also tested on the training data. 
This gives us some idea of how well the models predict the training data.

However, This does not test the ability of the models to generalize to unseen data. 
It is very likely that the local models, which are built on a relatively small amount of local data, are overfitting.

Therefore, we will retest these models in the following way: the first nine years of data for each stock will be used to train the models, and the last year will serve as the validation data.
So we have a 90/10 training/validation split.

In [1]:
from stock_utils import *

# Validate models over the last time period

In [2]:
df = pd.DataFrame()
df = df.from_csv('stock_data/nvda.csv')

daily_movements = get_price_movement_percentages(df)
movement_categories = categorize_movements(daily_movements, n_cats=4)

period_len = int(len(daily_movements) / 5)
train_period = daily_movements[0:4*period_len]
valid_period = daily_movements[4*period_len+1:5*period_len]

train_movement_categories = categorize_movements(train_period)
valid_movement_categories = categorize_movements(valid_period)

In [3]:
train_two_day_movement_trends = get_trends(train_movement_categories, 2)
train_three_day_movement_trends = get_trends(train_movement_categories, 3)
valid_two_day_movement_trends = get_trends(valid_movement_categories, 2)
valid_three_day_movement_trends = get_trends(valid_movement_categories, 3)

In [4]:
train_local_two_day_probs = build_model_probabilities(train_movement_categories, train_two_day_movement_trends, 2)
train_local_three_day_probs = build_model_probabilities(train_movement_categories, train_three_day_movement_trends, 3)
valid_local_two_day_probs = build_model_probabilities(valid_movement_categories, valid_two_day_movement_trends, 2)
valid_local_three_day_probs = build_model_probabilities(valid_movement_categories, valid_three_day_movement_trends, 3)

In [5]:
## One day probabilities
base_model = build_model_probabilities(train_movement_categories, [], 1)
base_model = np.array([base_model, base_model, base_model, base_model])

In [6]:
error_total = 0
for i in range(len(valid_local_two_day_probs)):
    error_val = 1
    if (i == category2index['bd']) or (i == category2index['bg']):
        error_val = 2  ## Penalize double on missing big movements
    for j in range(len(valid_local_two_day_probs)):
        error_total += error_val * abs(valid_local_two_day_probs[i][j] - train_local_two_day_probs[i][j])

avg_error_per_trend = error_total / len(valid_local_two_day_probs)**2
avg_error_per_initial_category =  error_total / len(valid_local_two_day_probs)

In [7]:
def get_error(model_1_probabilities, model_2_probabilities, n_day_model):
    model_1_probabilities = np.array(model_1_probabilities)
    model_2_probabilities = np.array(model_2_probabilities)
    
    if model_1_probabilities.ndim != model_2_probabilities.ndim:
        raise ValueError('Both models must have the same dimensions')
    
    total_error = 0
    
    ## One day model
    if (n_day_model == 1):
        for i in range(np.size(model_1_probabilities, 0)):
            error_val = 0.6
            if (i == category2index['bd']) or (i == category2index['bg']):
                error_val = 1.8  ## Penalize tripple on missing big movements
            total_error += error_val * abs(model_1_probabilities[i] - model_2_probabilities[i])
            
    ## Two day model
    elif (n_day_model == 2 or n_day_model == 3):
        for i in range(np.size(model_1_probabilities, 0)):
            for i in range(np.size(model_1_probabilities, 1)):
                error_val = 0.6
                if (j % 4 == category2index['bd']) or (j % 4 == category2index['bg']):
                    error_val = 1.8  ## Penalize tripple on missing big movements
                total_error += error_val * abs(model_1_probabilities[i][j] - model_2_probabilities[i][j])
        
    #elif (n_day_model == 3):
     #   print('lol')
      #  total_error = 100000000000000
    else:
        total_error = 999999999999999
    
    avg_error = total_error / model_1_probabilities.size
    return avg_error, total_error

### Try the following:
#### One day model performance
 - Base model (local 1 day) v.s. 1 day validation
 - Base model (local 1 day) v.s. 2 day validation
 - Base model (local 1 day) v.s. 3 day validation

#### Two day model performance
 - Local 2 day model v.s. 2 day validation
 - Composite 2 day model v.s. 2 day validation
 - Local 2 day model v.s. 3 day validation
 - Composite 2 day model v.s. 3 day validation

#### Three day model performance
 - Local 3 day model v.s. 3 day validation
 - Composite 3 day model v.s. 3 day validation

In [13]:
def build_three_day_local_composite_ensemble(local_three_day, composite_three_day):
    return (local_three_day + composite_three_day) / 2

In [14]:
def build_three_day_mixture_probs(one_day_probs, two_day_probs, three_day_probs):
    three_day_mixture_probs = 3 * np.copy(three_day_probs)
    
    ## Add one day probs
    outer_range = int(np.size(three_day_probs) / np.size(one_day_probs))
    for i in range(outer_range):
        three_day_mixture_probs[i] += one_day_probs

    ## Add two day probs
    outer_range = int(np.size(three_day_probs) / np.size(two_day_probs))
    for i in range(outer_range):
        three_day_mixture_probs[4*i:4*(i+1)][:] += 2 * two_day_probs
                   
    return three_day_mixture_probs / 6 ## Weighted Average

In [15]:
g = glob.glob('stock_data/*.csv')

(train_all_two_day_trends, _, train_all_single_day_category_probs, train_all_movement_categories,
valid_all_two_day_trends, _, valid_all_single_day_category_probs, valid_all_movement_categories) = \
      get_train_valid_trends_all_stocks(1, 2, movement_category_types, n_cats=4)
    
(train_all_three_day_trends, _, _, _, valid_all_three_day_trends, _, _, _) = \
      get_train_valid_trends_all_stocks(1, 3, movement_category_types, n_cats=4)

train_composite_two_day_probs = build_model_probabilities(train_all_movement_categories, train_all_two_day_trends, 2)
train_composite_three_day_probs = build_model_probabilities(train_all_movement_categories, train_all_three_day_trends, 3)

for filename in g:
    ticker = ticker_from_csv(filename)
    df = pd.DataFrame()
    df = df.from_csv(filename)
    daily_movements = get_price_movement_percentages(df)

    period_len = int(len(daily_movements) / 5)
    train_period = daily_movements[0:4*period_len]
    valid_period = daily_movements[4*period_len+1:5*period_len]

    train_movement_categories = categorize_movements(train_period)
    valid_movement_categories = categorize_movements(valid_period)

    train_two_day_movement_trends = get_trends(train_movement_categories, 2)
    train_three_day_movement_trends = get_trends(train_movement_categories, 3)
    valid_two_day_movement_trends = get_trends(valid_movement_categories, 2)
    valid_three_day_movement_trends = get_trends(valid_movement_categories, 3)

    train_base_model = build_model_probabilities(train_movement_categories, [], 1)
    train_local_two_day_probs = build_model_probabilities(train_movement_categories, train_two_day_movement_trends, 2)
    train_local_three_day_probs = build_model_probabilities(train_movement_categories, train_three_day_movement_trends, 3)
    train_local_three_day_mixture_probs = \
          build_three_day_mixture_probs(np.asarray(train_base_model), 
                                        np.asarray(train_local_two_day_probs), 
                                        np.asarray(train_local_three_day_probs))
    valid_local_one_day_probs = build_model_probabilities(valid_movement_categories, [], 1)
    valid_local_two_day_probs = build_model_probabilities(valid_movement_categories, valid_two_day_movement_trends, 2)
    valid_local_three_day_probs = build_model_probabilities(valid_movement_categories, valid_three_day_movement_trends, 3)
                                           
    print('=-=-=-=-=-=-=-=-=-=-=-=-=-=-=')
    print('             ' + ticker)
    print('=-=-=-=-=-=-=-=-=-=-=-=-=-=-=')
    print()

    print('---------------------------------------------------')
    print(' One day model performance')
    print('---------------------------------------------------')
    print(' Base model v.s. 1 day validation model')
    avg_error, total_error = get_error(train_base_model, valid_local_one_day_probs, 1)
    print('     Average error: ' + '{0:.2f}'.format(avg_error))
    print()
    print(' Base model v.s. 2 day validation model')
    train_base_2 = [train_base_model, train_base_model, train_base_model, train_base_model]
    avg_error, total_error = get_error(train_base_2, valid_local_two_day_probs, 2)
    print('     Average error: ' + '{0:.2f}'.format(avg_error))
    print()
    print(' Base model v.s. 3 day validation model')
    train_base_3 = train_base_2 * 4
    avg_error, total_error = get_error(train_base_3, valid_local_three_day_probs, 3)
    print('     Average error: ' + '{0:.2f}'.format(avg_error))
    print()
    
    print('---------------------------------------------------')
    print(' Two day model performance')
    print('---------------------------------------------------')
    print(' Local 2 day model v.s. 2 day validation model')
    avg_error, total_error = get_error(train_local_two_day_probs, valid_local_two_day_probs, 2)
    print('     Average error: ' + '{0:.2f}'.format(avg_error))
    print()
    print(' Composite 2 day model v.s. 2 day validation model')
    avg_error, total_error = get_error(train_composite_two_day_probs, valid_local_two_day_probs, 2)
    print('     Average error: ' + '{0:.2f}'.format(avg_error))
    print()
    print(' Local 2 day model v.s. 3 day validation model')
    avg_error, total_error = get_error(train_local_two_day_probs * 4, valid_local_three_day_probs, 3)
    print('     Average error: ' + '{0:.2f}'.format(avg_error))
    print()
    print(' Composite 2 day model v.s. 3 day validation model')
    avg_error, total_error = get_error(train_composite_two_day_probs * 4, valid_local_three_day_probs, 3)
    print('     Average error: ' + '{0:.2f}'.format(avg_error))
    print()
    
    print('---------------------------------------------------')
    print(' Three day model performance')
    print('---------------------------------------------------')
    print(' Local 3 day model v.s. 3 day validation model')
    avg_error, total_error = get_error(train_local_three_day_probs, valid_local_three_day_probs, 3)
    print('     Average error: ' + '{0:.2f}'.format(avg_error))
    print()
    print(' Composite 3 day model v.s. 3 day validation model')
    avg_error, total_error = get_error(train_composite_three_day_probs, valid_local_three_day_probs, 3)
    print('     Average error: ' + '{0:.2f}'.format(avg_error))
    print()
    print(' Local Mixture 3 day model v.s. 3 day validation model')
    avg_error, total_error = get_error(train_local_three_day_mixture_probs, valid_local_three_day_probs, 3)
    print('     Average error: ' + '{0:.2f}'.format(avg_error))
    print()
    #print(' 3 day local/composite ensemble model v.s. 3 day validation model')
    #ensemble = build_three_day_local_composite_ensemble(np.asarray(train_local_three_day_probs),
    #                                                    np.asarray(train_composite_three_day_probs))
    #avg_error, total_error = get_error(ensemble, valid_local_three_day_probs, 3)
    #print('     Average error: ' + '{0:.2f}'.format(avg_error))
    #print()
    print()
    

=-=-=-=-=-=-=-=-=-=-=-=-=-=-=
             BAC
=-=-=-=-=-=-=-=-=-=-=-=-=-=-=

---------------------------------------------------
 One day model performance
---------------------------------------------------
 Base model v.s. 1 day validation model
     Average error: 0.06

 Base model v.s. 2 day validation model
     Average error: 0.11

 Base model v.s. 3 day validation model
     Average error: 0.28

---------------------------------------------------
 Two day model performance
---------------------------------------------------
 Local 2 day model v.s. 2 day validation model
     Average error: 0.15

 Composite 2 day model v.s. 2 day validation model
     Average error: 0.07

 Local 2 day model v.s. 3 day validation model
     Average error: 0.17

 Composite 2 day model v.s. 3 day validation model
     Average error: 0.18

---------------------------------------------------
 Three day model performance
---------------------------------------------------
 Local 3 day model v.s. 3 day

---------------------------------------------------
 Three day model performance
---------------------------------------------------
 Local 3 day model v.s. 3 day validation model
     Average error: 0.36

 Composite 3 day model v.s. 3 day validation model
     Average error: 0.34

 Local Mixture 3 day model v.s. 3 day validation model
     Average error: 0.35


=-=-=-=-=-=-=-=-=-=-=-=-=-=-=
             TSLA
=-=-=-=-=-=-=-=-=-=-=-=-=-=-=

---------------------------------------------------
 One day model performance
---------------------------------------------------
 Base model v.s. 1 day validation model
     Average error: 0.05

 Base model v.s. 2 day validation model
     Average error: 0.08

 Base model v.s. 3 day validation model
     Average error: 0.08

---------------------------------------------------
 Two day model performance
---------------------------------------------------
 Local 2 day model v.s. 2 day validation model
     Average error: 0.09

 Composite 2 day model 

## Notes and Interpretation
It looks to me like some stocks have strong lcoal trends that they tend to follow.  
Here are some stocks that seem to follow either the local or composite trend patterns:
 - BAC
 - SPY
 - FB
 - INTC

Some stocks do not seem to follow multi-day movement trends strongly. These stocks can be identified by a relatively high 
performance (low error) on the 1-day, 2-day, and 3-day verification models (or a similar error compared to their two and three day model counterparts).  
This makes sense, because if picking movements randomly from the daily probability distribution describes all the daily movements, then looking back one or two days won't provide you any useful information.
 - NVDA
 - NKE
 - MU
 - GOOG
 - TSLA
 
Here are some middle of the road stocks (Maybe two day model worked well, but not three day):
 - SBUX
 - AMD
 
### Ideas for further experimentation
 - Retest these stocks to see if they maybe follow a weekly or monthly pattern 
 - Test a hybrid model, and hope that the variance of the three day model and the bias of the one day model balance correctly (worked for TSLA and NKE, and to a lesser extent NVDA and MU)
 - Retest with different training split
 - Weight probability distributions from more recent periods more heavily, like the concept of EMA
 
#### and if all else fails...
We can create strategies from the stocks we expect to follow particular trends. There is no need to consider every individual stock. Like Buffett said, we can "wait for our pitch" and consider stocks that follow trends to be our circle of competence.

## Hybrid model

In [17]:
ticker

'INTC'

In [16]:
train_local_two_day_probs

[[0.14220183486238533,
  0.3211009174311927,
  0.3486238532110092,
  0.18807339449541285],
 [0.09946236559139784,
  0.36693548387096775,
  0.44086021505376344,
  0.09274193548387097],
 [0.09134615384615384,
  0.39302884615384615,
  0.41947115384615385,
  0.09615384615384616],
 [0.17511520737327188,
  0.33640552995391704,
  0.3640552995391705,
  0.12442396313364056]]